# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
weather_df=pd.read_csv('../WeatherPy/Output/weather_df.csv')
weather_df.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,butaritari,67,KI,1612048770,77,3.0707,172.7902,81.16,17.87
1,1,tasiilaq,90,GL,1612048770,35,65.6145,-37.6368,21.20,17.27
2,2,hermanus,3,ZA,1612048771,86,-34.4187,19.2345,66.99,9.48
3,3,bilma,17,NE,1612048771,30,18.6853,12.9164,58.64,3.58
4,4,huarmey,66,PE,1612048771,70,-10.0681,-78.1522,73.78,10.63


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
gmaps.configure(api_key=g_key)

In [46]:
locations= weather_df[['Lat', 'Lng']]


fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weather_df['Humidity'], 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [51]:
ideal_weather=weather_df.loc[(weather_df['Max Temp']>70)&(weather_df['Max Temp']<80) & (weather_df['Wind Speed']<10) & (weather_df['Cloudiness']==0),:]
ideal_weather

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
27,27,caravelas,0,BR,1612048775,81,-17.7125,-39.2481,76.89,7.11
58,58,birao,0,CF,1612048779,21,10.2849,22.7882,71.51,7.85
97,97,coihaique,0,CL,1612048786,30,-45.5752,-72.0662,71.60,8.05
113,113,saint-philippe,0,RE,1612048789,65,-21.3585,55.7679,78.80,4.61
149,149,touros,0,BR,1612048794,83,-5.1989,-35.4608,78.80,5.75
153,153,san cristobal,0,ML,1612048795,19,13.3034,-4.8956,72.21,9.13
213,213,ahipara,0,NZ,1612048807,48,-35.1667,173.1667,70.48,5.64
258,258,san ciro de acosta,0,ML,1612048816,19,13.3034,-4.8956,72.21,9.13
309,309,nelson,0,NZ,1612048542,43,-41.2833,173.2833,75.99,1.01
337,337,bara,0,NG,1612048831,23,10.3744,10.7288,70.97,7.63


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [75]:
hotel_df=ideal_weather.loc[:,['City', 'Country', 'Lat', 'Lng']]
hotel_df['Hotel Name']=''
#hotel_df

params = {
    "radius": 5000,
    "type": "lodging",
    "keyword": "lodging",
    "key": g_key  
}

url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'

for index, row in hotel_df.iterrows():

    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    country = row['Country']
    params['location'] = f'{lat},{lng}'
    response = requests.get(url, params=params)
    hotel_name = response.json()

    try:
        print('Found a place to stay...keep.')
        hotel_df.loc[index, 'Hotel Name'] = hotel_name['results'][0]['name']
    except (KeyError, IndexError):
        print('Missing value.......skipping.')



Found a place to stay...keep.
Found a place to stay...keep.
Missing value.......skipping.
Found a place to stay...keep.
Found a place to stay...keep.
Missing value.......skipping.
Found a place to stay...keep.
Found a place to stay...keep.
Found a place to stay...keep.
Found a place to stay...keep.
Found a place to stay...keep.
Found a place to stay...keep.
Found a place to stay...keep.
Found a place to stay...keep.
Found a place to stay...keep.
Found a place to stay...keep.
Found a place to stay...keep.
Found a place to stay...keep.
Missing value.......skipping.
Found a place to stay...keep.
Found a place to stay...keep.
Missing value.......skipping.
Found a place to stay...keep.
Found a place to stay...keep.


In [81]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
27,caravelas,BR,-17.7125,-39.2481,Mar Doce de Abrolhos
58,birao,CF,10.2849,22.7882,
97,coihaique,CL,-45.5752,-72.0662,Abriga Bed and Breakfast
103,beloha,MG,-25.1667,45.0500,
113,saint-philippe,RE,-21.3585,55.7679,Les Embruns Du Baril
149,touros,BR,-5.1989,-35.4608,Hotel Vila Gale Touros
153,san cristobal,ML,13.3034,-4.8956,Hôtel Teriya
166,brandon,US,27.9378,-82.2859,Behind the Fence Bed and Breakfast
213,ahipara,NZ,-35.1667,173.1667,The Huts
253,miraflores,MX,19.2167,-98.8083,Hotel Mansión


In [82]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [83]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))